# Web Application of CrysPy 

By this application you can make refinement of polarized neutron diffraction data.

Calculations are done remotly without any necessety to install library on your personal machine.

Uppload tha file to run the refinement procedure.

In [1]:
# !pip3 install ipympl

In [2]:
%matplotlib widget
import ipywidgets
from ipywidgets import Output, Button, widgets, Layout
from IPython.display import HTML, clear_output, display
from base64 import b64encode

In [3]:
import cryspy

In [4]:
# global parameters
DDATA = {"rcif_obj": None, "file_rcif_name": "out.rcif"}
W_OUT = widgets.Output()

In [5]:
LAYOUT_50 = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    # border='solid 2px',
    align_items='stretch',
    width='49%'
)
LAYOUT_100 = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between',
    # border='solid 2px',
    align_items='stretch',
    width='98%',
    height="400px",
)

In [6]:
# page intro
def create_page_intro():
    widget_fo = widgets.FileUpload(
        description = "Upload RCIF",
        accept='.rcif',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False,  # True to accept multiple files upload else False
    )

    def change_output(ddata):
        s_cont = ddata["new"][0]["content"].tobytes().decode("ASCII")
        DDATA["rcif_obj"] = cryspy.str_to_globaln(s_cont)
        W_OUT.clear_output()
        with W_OUT:
            print(ddata["new"][0]["name"])
            DDATA["file_rcif_name"]=ddata["new"][0]["name"]
            print("File was uploaded")
        add_page_object_to_tab()
    
    widget_fo.observe(change_output, names="value")


    def on_button_clicked(b):
        DDATA["rcif_obj"] = cryspy.str_to_globaln("global_")
        W_OUT.clear_output()
        with W_OUT:
            print("Empty file was created")    
        add_page_object_to_tab()
        

    # button = widgets.Button(description="Empty object")
    # button.on_click(on_button_clicked)
    
    
    def trigger_download(text, filename, kind='text/json'):
        # see https://developer.mozilla.org/en-US/docs/Web/HTTP/Basics_of_HTTP/Data_URIs for details
        content_b64 = b64encode(text.encode()).decode()
        data_url = f'data:{kind};charset=utf-8;base64,{content_b64}'
        js_code = f"""
            var a = document.createElement('a');
            a.setAttribute('download', '{filename}');
            a.setAttribute('href', '{data_url}');
            a.click()
        """
        with W_OUT:
            clear_output()
            display(HTML(f'<script>{js_code}</script>'))
    
    btn_download = Button(description='Download File')
    
    def download_rcif_file(e=None):
        file_name = DDATA["file_rcif_name"]
        if DDATA["rcif_obj"] is not None:
            trigger_download(DDATA["rcif_obj"].to_cif(), file_name, kind='text/plain')
        else:
            with W_OUT:
                print("File is not specified")
        
    btn_download.on_click(download_rcif_file)
        
    items = [widget_fo, btn_download, ] # button,
    page = widgets.Box(items, layout=Layout(justify_content="center"))
    return page

In [11]:
# page object
def create_page_object():
    if DDATA["rcif_obj"] is None:
        page = widgets.Box(items)
        return page

    l_data_item_name = [item.get_name() for item in DDATA["rcif_obj"].items]
    
    select_data_item = widgets.Select(
        options=l_data_item_name,
        description='data items:',
        disabled=False,
        layout=LAYOUT_50,
        )
    select_loop_item = widgets.Select(
        options=[],
        description='loop items:',
        disabled=True,
        layout=LAYOUT_50,
        )

    cif_text = widgets.Textarea(
        value='',
        placeholder='Type or drop cif objet',
        description='CIF:',
        disabled=True,
        layout=LAYOUT_100,
    )

    button = widgets.Button(description="Save cif", layout=Layout(justify_content="center"))

    
    def func_loop(dinfo_data_item):
        data_index = select_data_item.index
        data_item = DDATA["rcif_obj"].items[data_index]
        W_OUT.clear_output()
        with W_OUT:
            for fig, ax in data_item.plots():
                display(fig)
            display(HTML(data_item.report_html()))
        if isinstance(data_item, cryspy.DataN):
            l_loop_item_name = [item.get_name() for item in data_item]
            select_loop_item.disabled = False
            select_loop_item.options = l_loop_item_name
            if len(l_loop_item_name) > 0:
                loop_item = data_item[select_loop_item.index]
                if loop_item is not None:
                    cif_text.value = loop_item.to_cif()
                    cif_text.disabled = False
                else:
                    cif_text.value = ""
                    cif_text.disabled = True
            else:
                cif_text.value = ""
                cif_text.disabled = True
        else:
            cif_text.value = data_item.to_cif()
            select_loop_item.options = ()
            select_loop_item.disabled = True
            cif_text.disabled = False
        return
    
    select_data_item.observe(func_loop, names="index")
    
    def show_cif(dinfo_loop_item):
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        loop_item = l_data_item[loop_index]
        cif_text.value = loop_item.to_cif()
        cif_text.disabled = False
        loop_item_report = loop_item.report()
        
        W_OUT.clear_output()
        with W_OUT:
            for fig, ax in loop_item.plots():
                display(fig)
            display(HTML(loop_item.report_html()))

        
    select_loop_item.observe(show_cif, names="index")
    
    def save_cif(b):        
        data_index = select_data_item.index
        l_data_item = DDATA["rcif_obj"].items[data_index]
        loop_index = select_loop_item.index
        
        globaln_new = cryspy.str_to_globaln(cif_text.value)
        if len(globaln_new.items) != 0:
            loop_item_new = globaln_new.items[0]
            if isinstance(loop_item_new, (cryspy.LoopN, cryspy.ItemN)):
                l_data_item.items.pop(loop_index)
                l_data_item.items.insert(loop_index, loop_item_new)
                    
    button.on_click(save_cif)
    
    
    item_v = [widgets.HBox([select_data_item, select_loop_item, ]), cif_text, button]
    page = widgets.VBox(item_v)
    
    return page

In [12]:

def run_fitting(b):
    W_OUT.clear_output()
    with W_OUT:
        print("Calculations started")
        cryspy.rhochi_rietveld_refinement(DDATA["rcif_obj"])
        print("Calculations finished")
        for fig, ax in DDATA["rcif_obj"].plots():
            display(fig)
        add_page_object_to_tab()
    
def run_chi_sq(b):
    W_OUT.clear_output()
    with W_OUT:
        print("Calculations started")
        cryspy.rhochi_no_refinement(DDATA["rcif_obj"])
        print("Calculations finished")
        for fig, ax in DDATA["rcif_obj"].plots():
            display(fig)
        add_page_object_to_tab()
        
def run_mempy_sd(b):
    W_OUT.clear_output()
    with W_OUT:
        print("Calculations started")
        cryspy.mempy_spin_density_reconstruction(DDATA["rcif_obj"])
        print("Calculations finished")
        for fig, ax in DDATA["rcif_obj"].plots():
            display(fig)
        add_page_object_to_tab()
    
def run_mempy_md(b):
    W_OUT.clear_output()
    with W_OUT:
        print("Calculations started")
        cryspy.mempy_magnetization_density_reconstruction(DDATA["rcif_obj"])
        print("Calculations finished")
        for fig, ax in DDATA["rcif_obj"].plots():
            display(fig)
        add_page_object_to_tab()
    
    
button_fitting = widgets.Button(description="Fitting")
button_calculations = widgets.Button(description="Chi2")
button_mempy_sd = widgets.Button(description="Maximum Entropy (Spin density)")
button_mempy_md = widgets.Button(description="Maximum Entropy (Magnetization density)")

items = [button_fitting, button_calculations, button_mempy_sd, button_mempy_md]

button_fitting.on_click(run_fitting)
button_calculations.on_click(run_chi_sq)
button_mempy_sd.on_click(run_mempy_sd)
button_mempy_md.on_click(run_mempy_md)


page_methods = widgets.HBox(items, layout=Layout(justify_content="center"))

In [13]:
tab = widgets.Tab()
page_intro = create_page_intro()
tab.children = (page_intro, )
tab.titles = ("Intro", )
display(tab, page_methods, W_OUT)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', '…

In [14]:
def add_page_object_to_tab():
    page_object = create_page_object()
    if len(tab.children) == 1:
        tab.children += (page_object, )
        tab.titles = tab.titles[:-1] + ("Object", )
    else:
        tab.children = tab.children[:1]+(page_object, )+tab.children[2:]
    tab.selected_index = 1


In [ ]:
# select = widgets.Select(
#     options=['Linux', 'Windows', 'macOS'],
#     value='macOS',
#     # rows=10,
#     description='OS:',
#     disabled=False
# )
# download_output = Output()
# def change_output(value: str):
#     download_output.clear_output()
#     with download_output:
#         print(value["new"])
# select.observe(change_output, names="value")
# display(select, download_output)

In [ ]:
# widgets.Dropdown(
#     options=['1', '2', '3'],
#     value='2',
#     description='Number:',
#     disabled=False,
# )

In [ ]:
# widgets.Combobox(
#     # value='John',
#     placeholder='Choose Someone',
#     options=['Paul', 'John', 'George', 'Ringo'],
#     description='Combobox:',
#     ensure_option=True,
#     disabled=False
# )

In [ ]:
# from IPython.display import FileLink
# filename = "fff.rcif"
# FileLink(filename)